<a href="https://colab.research.google.com/github/arnavwinner/ML_Project/blob/main/Face_Recognition_Attendance_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# from google.colab import files
# files.upload()  # Upload the kaggle.json file

# # Move the uploaded kaggle.json to the appropriate directory
# !mkdir -p ~/.kaggle
# !mv kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

from google.colab import files

# Use the 'files.upload()' function to upload your dataset
uploaded = files.upload()

# Provide the file path to the uploaded dataset
file_path = '/content/' + list(uploaded.keys())[0]

# You can verify the file is uploaded using the following line
print("Uploaded file:", file_path)


Saving image_dataset.zip to image_dataset.zip
Uploaded file: /content/image_dataset.zip


In [4]:
# !pip install deepface tensorflow kaggle
import zipfile

# Specify the path to the uploaded ZIP file
zip_file_path = '/content/' + list(uploaded.keys())[0]

# Specify the directory where you want to extract the contents
extracted_dir_path = '/content/my_dataset/'

# Create the target directory if it doesn't exist
import os
os.makedirs(extracted_dir_path, exist_ok=True)

# Extract the ZIP file to the target directory
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir_path)

# List the contents of the extracted directory
extracted_files = os.listdir(extracted_dir_path)
print("Extracted files:", extracted_files)

Extracted files: ['image_dataset']


In [ ]:
# !kaggle datasets download -d jessicali9530/lfw-dataset
# !unzip -q lfw-dataset.zip -d dataset  # Unzip the dataset to a 'dataset' directory


 99% 111M/112M [00:01<00:00, 91.3MB/s] 
100% 112M/112M [00:01<00:00, 79.8MB/s]


In [5]:
# !ls /content/dataset/lfw-deepfunneled/lfw-deepfunneled
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 12.7 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116933 sha256=1d69ded2f61b2589c1b6c1e83cce0f06c3f763eb22b680e4845cccfb1697f493
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [1]:
# from deepface import VGGFace
from deepface import DeepFace
from deepface.basemodels import VGGFace
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# Define the number of classes in your dataset
num_classes = 14  # Replace with the actual number of classes in your dataset

# Define batch size and number of epochs
batch_size = 1
num_epochs = 15

# Specify the path to your dataset
dataset_path = "/content/my_dataset/data_set_train"  # Replace with the actual path

# Load the VGGFace model without the top classification layer
base_model = VGGFace.loadModel()

# Add custom top layers for classification
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

# ... (rest of your code)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

# Data Augmentation (optional)
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Load your data using flow_from_directory
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Use this to specify 'training' subset
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Use this to specify 'validation' subset
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    epochs=num_epochs
)

# Save the model
model.save("trained_face_recognition_model.h5")

# Evaluate the model
test_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Use this to specify 'validation' subset for testing
)

model.evaluate(test_generator)
# print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
############################################################################
# from keras.applications import VGG16
# from keras.layers import Dense, Flatten
# from keras.models import Model
# from keras.optimizers import Adam
# from keras.preprocessing.image import ImageDataGenerator

# # Define the number of classes in your dataset
# num_classes = 14  # Replace with the actual number of classes in your dataset

# # Define batch size and number of epochs
# batch_size = 32
# num_epochs = 10

# # Specify the path to your dataset
# dataset_path = "/content/my_dataset/data_set_train"  # Replace with the actual path

# # Load the VGG16 model without the top classification layer
# base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# # Add custom top layers for classification
# x = Flatten()(base_model.output)
# x = Dense(1024, activation='relu')(x)
# x = Dense(512, activation='relu')(x)
# output = Dense(num_classes, activation='softmax')(x)

# model = Model(inputs=base_model.input, outputs=output)

# # Compile the model
# model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# # Data Augmentation (optional)
# datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True
# )

# # Load your data using flow_from_directory
# train_generator = datagen.flow_from_directory(
#     dataset_path,
#     target_size=(224, 224),
#     batch_size=batch_size,
#     class_mode='categorical',
#     subset='training'  # Use this to specify 'training' subset
# )

# validation_generator = datagen.flow_from_directory(
#     dataset_path,
#     target_size=(224, 224),
#     batch_size=batch_size,
#     class_mode='categorical',
#     subset='validation'  # Use this to specify 'validation' subset
# )

# # Train the model
# history = model.fit(
#     train_generator,
#     steps_per_epoch=len(train_generator),
#     validation_data=validation_generator,
#     validation_steps=len(validation_generator),
#     epochs=num_epochs
# )

# # Save the model
# model.save("trained_face_recognition_model.h5")

# # Evaluate the model
# test_generator = datagen.flow_from_directory(
#     dataset_path,
#     target_size=(224, 224),
#     batch_size=batch_size,
#     class_mode='categorical',
#     subset='validation'  # Use this to specify 'validation' subset for testing
# )

# test_loss, test_accuracy = model.evaluate(test_generator)
# print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


ModuleNotFoundError: ignored

In [ ]:
# import os
# from tensorflow.keras.layers import Dense, Flatten, Input
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from keras_arcface import ArcFace

# # Define the number of classes in your dataset
# num_classes = 15  # Replace with the actual number of classes in your dataset

# # Define batch size and number of epochs
# batch_size = 32
# num_epochs = 20

# # Specify the path to your dataset
# dataset_path = "/content/my_dataset/data_set_train"  # Replace with the actual path

# # Load the ResNet-50 base model
# input_shape = (224, 224, 3)
# input_tensor = Input(shape=input_shape)
# base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)

# # Add custom top layers for classification with ArcFace loss
# x = Flatten()(base_model.output)
# x = Dense(1024, activation='relu')(x)
# x = Dense(512, activation='relu')(x)
# output = Dense(num_classes, activation='softmax')(x)

# model = Model(inputs=base_model.input, outputs=output)

# # Add ArcFace layer
# output = ArcFace(num_classes)(model.layers[-2].output)

# # Create a new model with the ArcFace layer
# arcface_model = Model(inputs=model.input, outputs=output)

# # Compile the model with the ArcFace loss
# arcface_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# # Data Augmentation (optional)
# datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True
# )

# # Load your data using flow_from_directory
# train_generator = datagen.flow_from_directory(
#     dataset_path,
#     target_size=input_shape[:2],
#     batch_size=batch_size,
#     class_mode='categorical',
#     subset='training'  # Use this to specify the 'training' subset
# )

# validation_generator = datagen.flow_from_directory(
#     dataset_path,
#     target_size=input_shape[:2],
#     batch_size=batch_size,
#     class_mode='categorical',
#     subset='validation'  # Use this to specify the 'validation' subset
# )

# # Train the model
# history = arcface_model.fit(
#     train_generator,
#     steps_per_epoch=len(train_generator),
#     validation_data=validation_generator,
#     validation_steps=len(validation_generator),
#     epochs=num_epochs
# )

# # Save the model
# arcface_model.save("trained_face_recognition_model.h5")

# # Evaluate the model
# test_generator = datagen.flow_from_directory(
#     dataset_path,
#     target_size=input_shape[:2],
#     batch_size=batch_size,
#     class_mode='categorical',
#     subset='validation'  # Use this to specify the 'validation' subset for testing
# )

# test_loss, test_accuracy = arcface_model.evaluate(test_generator)
# print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
